# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

17:54:39.913157 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
17:54:39.915533 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
17:54:39.915627 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
17:54:39.923852 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
17:54:39.938940 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7fb4d540ce70>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
# ANSI escape codes for colors
CRESET    = '\33[0m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CVIOLET = '\33[35m'

import datetime
t = lambda: datetime.datetime.now().strftime("%H:%M:%S.%f")


def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    elif tag == "server":
        color = CVIOLET
    else:
        color = CGREEN
    print(f"{color}{t()} {message}{CRESET}", flush=True)
 
#cprint = lambda tag, message: None    # Uncomment this to disable Python logging
class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
{"c": client_tiface, "s": server_tiface}

{'c': <__main__.LoggingTransportInterface at 0x7fb4d55cf970>,
 's': <__main__.LoggingTransportInterface at 0x7fb4d55cc070>}

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7fb4d413c500>,
 's': <FilterInterface at 0x7fb4d413c3d0>}

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

17:54:40.023651 LTI(server)0: got transport notifications: []17:54:40.024265 LTI(client)0: got transport notifications: []
17:54:40.025574 [INFO ] - [F] About to send pings to servers: []



In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation for Pings

### As a client, add server (both our local and Chococat) as ping endpoints

In [10]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

17:54:40.223561 [INFO ] - [F] About to send pings to servers: []
17:54:40.423629 [INFO ] - [F] About to send pings to servers: []


'157.230.134.224'

# !!! COMMENTED OUT server_ept BELOW !!!

In [11]:
#server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[choco_ept])  # endpoints=[server_ept, choco_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(157.230.134.224:2016)] }

In [12]:
await client_fiface.command_response(filter_cmd)

17:54:40.612191 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(157.230.134.224:2016)] }


Accepted

### Here be dragons

Unblock the Python async event loop by running some random async code. IDK why?

**Anyway,** I just copied this shit from StackOverflow with slight modifications for Jupyter (event loop already running): https://stackoverflow.com/a/53420574

In [13]:
import time
import asyncio

async def sleeper():
    print(f'Time: {time.time() - start:.2f}')
    await asyncio.sleep(1)
async def sum(name, numbers):
    total = 0
    for number in numbers:
        print(f'Task {name}: Computing {total}+{number}')
        await sleeper()
        total += number
    print(f'Task {name}: Sum = {total}\n')

start = time.time()
loop = asyncio.get_event_loop()
tasks = [
    loop.create_task(sum("A", [1, 2])),
    loop.create_task(sum("B", [1, 2, 3])),
]
done, pending = await asyncio.wait(tasks)
assert len(pending) == 0

end = time.time()
print(f'Time: {end-start:.2f} sec')

Task A: Computing 0+117:54:40.623944 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]

Time: 0.00
Task B: Computing 0+1
Time: 0.00
17:54:40.625605 LTI(client)9 skipped 8: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 15316352245136124142 } }] }
17:54:40.628500 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 15316352245136124142 } }] }
17:54:40.630096 LTI(client)10: got transport_rsp back from Transport layer: TransportRsp::Accepted
17:54:40.631894 [TRACE] - [F<-T,R] Command Accepted
17:54:40.655922 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 15316352245136124142 }, server_versio

17:54:41.479310 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 6286671867149777893 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:54:41.479374 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(23)
17:54:41.579658 LTI(client)28: got transport notifications: []
17:54:41.624057 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:41.625347 LTI(client)29 skipped 0: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 2582219751946802826 } }] }
Task A: Computing 1+2
Time: 1.00
Task B: Computing 1+2
Time: 1.00
17:54:41.626777 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: Pr

17:54:42.428770 [TRACE] - [F<-T,R] Command Accepted
17:54:42.451010 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 2388099686841935519 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:54:42.500748 LTI(client)47: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 2388099686841935519 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } }]
17:54:42.501628 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 2388099686841935519 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:54:42.501655 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(35)
17:54:42.602438 LTI(client)48: got transport notifications: []
17:54:42.624031 [INFO ] - [F] About to send pings to servers: [Endpoint(1

17:54:43.428092 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 6 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 13767917458042532214 } }] }
17:54:43.428423 LTI(client)66: got transport_rsp back from Transport layer: TransportRsp::Accepted
17:54:43.431452 [TRACE] - [F<-T,R] Command Accepted
17:54:43.452589 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 13767917458042532214 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:54:43.525878 LTI(client)67: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 13767917458042532214 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } }]
17:54:43.526933 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took pac

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

In [14]:
time.sleep(3.0)
server_notifications = server_fiface.get_notifications()
server_notifications

17:54:43.649037 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 15425400246305596496 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:54:43.732250 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 15425400246305596496 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
17:54:43.732278 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(49)
17:54:43.823555 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:44.023473 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:44.223331 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:44.423923 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:44.624219 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:44

[]

In [15]:
client_notifications = client_fiface.get_notifications()
client_notifications

[PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: Some(23), server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: Some(25), server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 P

Sleeping again....

In [16]:
time.sleep(10.0)
client_notifications = client_fiface.get_notifications()
client_notifications

17:54:46.823738 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:47.023371 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:47.223731 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:47.423922 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:47.623502 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:47.823917 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:48.023774 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:48.224402 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:48.423970 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:48.623895 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
17:54:48.686937 [INFO ] - [F<-T,N] Endpoint(157.23

[]

In [17]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

17:54:56.679108 LTI(client)199 skipped 128: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 7 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 13049980209706646729 } }] }17:54:56.679960 LTI(client)199: got transport notifications: []
17:54:56.678844 [TRACE] - [F<-A,C] New command: ClearPingEndpoints
17:54:56.678865 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(157.230.134.224:2016)]

17:54:56.684238 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 7 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 13049980209706646729 } }] }


Accepted

17:54:56.687917 LTI(client)201: got transport_rsp back from Transport layer: TransportRsp::Accepted


In [18]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

17:54:56.708693 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 13049980209706646729 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }


### Now shutdown

In [ ]:
sleep(1)
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))